<a href="https://colab.research.google.com/github/valievav/ML-projects/blob/main/NER_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install spacy

In [2]:
# download model
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 14.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
!pip install spacy-lookups-data

In [4]:
# JIz-hiRrZ2g&t=18s

import spacy

# Check NER model results on test data

In [5]:
text = [
    'Nessa lives in Kyiv',
    'Chris Cornell was part of the Soundgarden before Audioslave',
    'Sakura blossoms in Japan in spring',
    'OpenAI released CharGPT Pro version in December 2024',
    'Lia speaks both Ukrainian and English',
    'R2D2 can only communicate in Binary',
]

# use existing model
nlp = spacy.load('en_core_web_md')

# get entities that can be recognized
ner_labels = nlp.get_pipe('ner').labels
ner_labels

('CARDINAL',
 'DATE',
 'EVENT',
 'FAC',
 'GPE',
 'LANGUAGE',
 'LAW',
 'LOC',
 'MONEY',
 'NORP',
 'ORDINAL',
 'ORG',
 'PERCENT',
 'PERSON',
 'PRODUCT',
 'QUANTITY',
 'TIME',
 'WORK_OF_ART')

In [6]:
docs = [nlp(x) for x in text]

for doc in docs:
  entities = []
  for ent in doc.ents:
    entities.append([ent.text, ent.label_])

  print(entities)


[['Nessa', 'PERSON'], ['Kyiv', 'GPE']]
[['Chris Cornell', 'PERSON'], ['the Soundgarden before Audioslave', 'GPE']]
[['Sakura', 'PERSON'], ['Japan', 'GPE'], ['spring', 'DATE']]
[['CharGPT Pro', 'PRODUCT'], ['December 2024', 'DATE']]
[['Lia', 'PERSON'], ['Ukrainian', 'NORP'], ['English', 'LANGUAGE']]
[]


# Fine-tune existing NER model on custom data

In [7]:
import random
from spacy.util import minibatch
from spacy.training.example import Example

train_data = [
    ('What is the price of 10 bananas?', {'entities': [(21, 23, 'QUANTITY'), (24, 31, 'PRODUCT')]}),
    ('How much does 5 apples cost?', {'entities': [(15, 16, 'QUANTITY'), (17, 23, 'PRODUCT')]}),
    ('Can you tell me the price of 2 oranges?', {'entities': [(29, 30, 'QUANTITY'), (31, 38, 'PRODUCT')]}),
    ('I want to buy 1 watermelon. How much is it?', {'entities': [(14, 15, 'QUANTITY'), (16, 26, 'PRODUCT')]}),
    ('What would 3 pineapples cost?', {'entities': [(11, 12, 'QUANTITY'), (13, 23, 'PRODUCT')]}),
    ('Give me the price for 12 mangoes.', {'entities': [(21, 23, 'QUANTITY'), (24, 31, 'PRODUCT')]}),
    ('How much for 8 avocados?', {'entities': [(13, 14, 'QUANTITY'), (15, 23, 'PRODUCT')]}),
    ('I need the cost of 6 lemons.', {'entities': [(19, 20, 'QUANTITY'), (21, 27, 'PRODUCT')]}),
    ('Could you tell me how much 4 pears are?', {'entities': [(26, 27, 'QUANTITY'), (28, 33, 'PRODUCT')]}),
    ('Price check on 7 strawberries please.', {'entities': [(14, 15, 'QUANTITY'), (16, 28, 'PRODUCT')]}),
    ('How much would 9 peaches be?', {'entities': [(15, 16, 'QUANTITY'), (17, 24, 'PRODUCT')]}),
    ('What’s the rate of 11 grapes?', {'entities': [(19, 21, 'QUANTITY'), (22, 28, 'PRODUCT')]}),
    ('Can I know the price of 3 cherries?', {'entities': [(25, 26, 'QUANTITY'), (27, 35, 'PRODUCT')]}),
    ('How expensive are 13 blueberries?', {'entities': [(18, 20, 'QUANTITY'), (21, 32, 'PRODUCT')]}),
    ('Do you know the cost of 2 melons?', {'entities': [(24, 25, 'QUANTITY'), (26, 32, 'PRODUCT')]}),
    ('Please tell me the price of 5 plums.', {'entities': [(28, 29, 'QUANTITY'), (30, 35, 'PRODUCT')]}),
    ('How much is 14 kiwis?', {'entities': [(12, 14, 'QUANTITY'), (15, 20, 'PRODUCT')]}),
    ('What’s the total for 6 papayas?', {'entities': [(18, 19, 'QUANTITY'), (20, 27, 'PRODUCT')]}),
    ('I’d like the price of 1 coconut.', {'entities': [(22, 23, 'QUANTITY'), (24, 31, 'PRODUCT')]}),
    ('Tell me how much 10 apricots cost.', {'entities': [(18, 20, 'QUANTITY'), (21, 29, 'PRODUCT')]}),
]

nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [8]:
# use existing model
nlp = spacy.load('en_core_web_md')

# make sure ner pipeline is in the model
if 'ner' not in nlp.pipe_names:
  ner = nlp.add_pipe('ner')
else:
  ner = nlp.get_pipe('ner')

# add unknown labels to the model
for _, ent_data in train_data:
  for ent in ent_data['entities']:
    label = ent[2]
    if label not in ner.labels:
      ner.add_label[label]

# disable all pipelines but ner and train it
epochs = 50
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']

with nlp.disable_pipes(*other_pipes):
  # optimizer = nlp.begin_training()  # training a new blank model
  optimizer = nlp.create_optimizer()  # fine-tuning already trained model

  for epoch in range(epochs):
    random.shuffle(train_data)
    losses = {}
    batches = minibatch(train_data, size=2)
    for batch in batches:
      examples = []
      for text, ent_data in batch:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, ent_data)
        examples.append(example)
      nlp.update(examples, drop=0.5, losses=losses)
    print(f'Epoch {epoch + 1}, losses {losses}')

# save model
nlp.to_disk('custom_ner_model')

trained_nlp = spacy.load('custom_ner_model')

test_text = [
    'How much for five avocados?',
    'Ten candies please, what''s the price?',
    'I''m intereseted in 5 granola bars, how much is it?',
]

for text in test_text:
  doc = trained_nlp(text)
  print(f'Text {text}')
  print(f'Entities {[(ent.text, ent.label_) for ent in doc.ents]}')
  print('---')

/usr/local/lib/python3.11/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Give me the price for 12 mangoes." with entities "[(21, 23, 'QUANTITY'), (24, 31, 'PRODUCT')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Could you tell me how much 4 pears are?" with entities "[(26, 27, 'QUANTITY'), (28, 33, 'PRODUCT')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Can I know the price of 3 cherrie

Epoch 1, losses {'ner': 47.46584997915339}
Epoch 2, losses {'ner': 39.21012109524224}
Epoch 3, losses {'ner': 19.989931263672013}
Epoch 4, losses {'ner': 6.828884455976867}
Epoch 5, losses {'ner': 8.851253650917455}
Epoch 6, losses {'ner': 6.477666019772375}
Epoch 7, losses {'ner': 6.211562203178126}
Epoch 8, losses {'ner': 7.979154751915947}
Epoch 9, losses {'ner': 9.690073171735792}
Epoch 10, losses {'ner': 3.9692885197029812}
Epoch 11, losses {'ner': 4.007753156503148}
Epoch 12, losses {'ner': 3.360966258075799}
Epoch 13, losses {'ner': 3.5716328733559704}
Epoch 14, losses {'ner': 3.0858315878958402}
Epoch 15, losses {'ner': 3.0867949062685596}
Epoch 16, losses {'ner': 0.32949128447640313}
Epoch 17, losses {'ner': 3.6022165885438047}
Epoch 18, losses {'ner': 2.3703260871525793}
Epoch 19, losses {'ner': 2.045569788099434}
Epoch 20, losses {'ner': 4.821908866691978}
Epoch 21, losses {'ner': 4.801777871131605}
Epoch 22, losses {'ner': 1.1611619480815376}
Epoch 23, losses {'ner': 3.5395